<h1><b>Statistique en Bioinformatique : </b> TME3 </h1>
<br>
L’objectif de ce TME est:
<br>
<ul>
<li> objectif 1: comprendre les principales étapes de BLAST et PSIBLAST, </li>
<li> objectif 2: comprendre et comparer les algoritimes de l'alignement progressive </li>
</ul>
<br>
<div class="alert alert-warning" role="alert" style="margin: 10px">
<p><b>Soumission</b></p>
<ul>
<li>Soumettre sur moodle </li>
</ul>
</div>


Nom etudiant 1 : ANTON Christine
<br>
Nom etudiant 2 : NICOLAS Elise
<br>

<h3>Section 1: BLAST</h3>

<b> Exercice 1 </b>: À partir de l'application BLAST, disponible via le site de <a href="http://blast.ncbi.nlm.nih.gov/">NCBI</a>, cliquez sur protein Blast. Faites un BLAST de la séquence fasta/seq1.fasta. Expliquez le résultat graphique, la couleur de chaque hit et expliquez le tableau de résultat qu'est-ce que c'est chaque colonne?
<br>


<font color="red"> Réponse : </font>
<font color="blue"> 
<br>
The graphical result of the alignments shows the distribution of the top 120 BLAST hits on 100 subject sequences. The color of each hit reflects the quality of the hit : red -> high score (>=200); pink -> medium score (80–200); green -> low score (50–80); blue -> even lower score (40–50); black -> very low score (<40). The length of the bar reflects the extent of the alignment relative to the query sequence.
<br>
For seq1.fasta, we observe around 40 red hits (score >= 200) of the same length as the query sequence, along with other shorter sequences with slightly lower alignment scores (between 50 and 200).
<br>
The results table lists the subject sequences that match our query sequence. The different columns are : Description (description of the protein sequence), Scientific name (name and species of the identified protein), Max score, Total score, Query cover (percentage of the query sequence covered by the alignment), E-value (probability of obtaining this result by chance (the closer it is to 0, the better)), Identity percentage, Length of sequence, and Accession (link to the complete sequence in the database).
</font>

<b>Exercice 2</b>: On s'intéresse à présent au gène FHIT chez l'homme (fragile histidine triad protein, identifiant P49789, télécharger du site  <a href="http://www.uniprot.org/"> Uniprot </a>). La protéine de ce gène est structurellement proche des galactose-1-phosphate uridylyltransferases. Faites un BLAST de ce gène contre la base SWISSPROT. Que constatez-vous?

<font color="red"> Réponse : </font>
<font color="blue">
Entièrement séquences rouges
</font>

<b>Exercice 3</b>: Implémenter l'étape 0 du algorithme BLAST (indexer la base de données), utilise w=5 et les séquences du fichier database.fasta.

In [39]:
# Imports

import numpy as np
from itertools import combinations, combinations_with_replacement


In [5]:
#variable initiation
database = 'database.fasta'
seqTestBlast = 'test.fasta'
input_blosum_f = 'BLOSUM62.txt'
aa = ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V', 'X']

#parameters
q = 21
gap = -5
w = 5

#For a simple test use:
database = 'databaseToy.fasta'
seqTestBlast = 'testToy.fasta'
w=3

In [6]:
#Read the test.fasta file
def read_fasta(input_f):
    """
    Input1 input_f: fasta file name
    Output1 arr_seqs: List of sequences
    REMARK : We keep only the sequences, identifiers are not necessary
    """
    arr_seqs = []
    with open(input_f, 'r') as in_f :
        for line in in_f :
            if not line.startswith ('>') : 
                arr_seqs.append(line.strip())
    return arr_seqs

In [47]:
def read_fasta_dict(input_f):
    """
    Input1 input_f: fasta file name
    Output1 arr_seqs: Dict of name_seq : sequences
    REMARK : We keep only the sequences, identifiers are not necessary
    """
    dict_seqs = dict()
    with open(input_f, 'r') as in_f :
        header = None
        for line in in_f :
            if line.startswith ('>') :
                header = line[1:]
                header = [char for char in list(header) if char !='\n']
                header = "".join(header)
                dict_seqs[header]=""
            else : 
                line = [char for char in list(line) if char !='\n']
                dict_seqs[header]+="".join(line).upper()
            
    return dict_seqs

In [48]:

def etape0(fileName, w):
    """
    Database indexation
    Input1 fileName: fasta file name
    Input2 w:  word size
    Output1 index: dictionary containing indexed words
    Output2 dicoSeq: dictionary containing database entire sequences
    """
    index={}
    dicoSeq = read_fasta_dict(fileName)
   
    for seq_name, seq in dicoSeq.items() : 
        for i in range(len(seq)-w+1):
            word = seq[i:i+w].upper()
            p = (seq_name, i)
            if (word) in index.keys():
                index[word].append(p)
            else : 
                index[word]=[p]
    return index,dicoSeq

index,dicoSeq = etape0(database, w)
print (index)
print (dicoSeq)
#{'AGH': [('s1', 0)], 'CGG': [('s3', 0)], 'GGH': [('s2', 1)]}
#{'s1': 'AGHR', 's2': 'CGGHR', 's3': 'CGGH'}

{'AGH': [('s1', 0)], 'GHR': [('s1', 1), ('s2', 2)], 'CGG': [('s2', 0), ('s3', 0)], 'GGH': [('s2', 1), ('s3', 1)], 'GHA': [('s3', 2)]}
{'s1': 'AGHR', 's2': 'CGGHR', 's3': 'CGGHA'}


<b>Exercice 4</b>: Chercher la séquence query test.fasta dans la base indexé construite précédemment, entendez l'alignement pour avoir un alignement sans gap et de score > 12. Retournez les quatre meilleurs alignements.


In [49]:
#read Blosum
def read_blosum_matrix(input_f) :
    """
    Input1 input_f : blosum file name
    Output1: a matrix of dimention [q-1, q-1]
    REMARK : You should eliminate the columns/rows B, Z and * from Blosum file
    """
    blosum = np.zeros((q-1,q-1))
    unwanted_aa = {'B', 'Z', 'X', '*'}
    first_line = True
    aa_list = []
    matrix_lines = []
    row_idx = 0
    
    with open(input_f, 'r') as in_f :
        for line in in_f :
            if line.startswith("#") or line.strip() == "" :
                continue
            
            if first_line :
                first_line = False 
                aa_list = line.split()
                continue

            parts = line.split()
            scores = list(map(int, parts[1:]))
            matrix_lines.append(scores)

    matrix = np.array(matrix_lines)
    for aa in unwanted_aa :
        ind = aa_list.index(aa)
        matrix = np.delete(matrix, ind, axis=0)
        matrix = np.delete(matrix, ind, axis=1)
        del aa_list[ind]

    blosum = matrix 
    
    return blosum

In [42]:
#read Blosum
def readBlosum(nameFile):
    """
    Read substitution matrix file
    Input1 fileName : blosum file name
    Output1 Blosum: a dictionary containing BLOSUM matrix, key is a pair of amino acids (alphabetic order) and value is the substitution value.
    REMARK : You could eliminate the last four columns and last four rows of Blosum file
    """
    Blosum={}

    # ReadBlossum into matrix
    B = read_blosum_matrix(nameFile)
    
    for i in range(len(B)):
        for j in range(len(B)):
            aa1=aa[i]
            aa2=aa[j]
            Blosum[aa1+aa2]=B[i,j]

    return Blosum

blosum = readBlosum(input_blosum_f)
print (blosum["AA"])
#4

4


In [50]:
def extendAln(pos1, pos2, sequence1, sequence2, matriceBlosum, scoremotif, w, ThrScore):
    """
    extend alignment left/right by keeping score above a threshold
    Input1 pos1 : position in the first sequence (Subject)
    Input2 pos2 : position in the second sequence (query)
    Input3 sequence1 : first sequence (Subject)
    Input4 sequence1 : second sequence (query)
    Input5 matriceBlosum: a dictionary containing BLOSUM matrix
    Input6 scoremotif: initial score
    Input7 w: word size
    Input8 ThrScore: threshold
    Output 1 finalScore: final score
    Output 2 algLength: alignment length
    Output 3 alnSeqs: Tuple containing aligned sequences
    """

    # Initialisation
    finalScore= 0
    algLength = 0
    alnSeqs = ()

    # Extend similar motif
    for i, j in zip (range(pos1,len(sequence1)), range(pos2,len(sequence2))):
        # calcul du nouveau score
        aa1=sequence1[i]
        aa2=sequence2[j]
        n_score = matriceBlosum[aa1+aa2]
        if n_score > ThrScore : 
            finalScore = scoremotif
        else : 
            scoremotif = n_score
            finalScore=scoremotif
            algLength+=1
            alnSeqs[0]+=aa1
            alnSeqs[1]+=aa2    
            

    # Calcul de score

    return [finalScore, algLength, alnSeqs]

res = extendAln(1, 1, dicoSeq['s2'], 'AGGHV', blosum, 20, 3, 18)
print (res)
#[20, 4, ('CGGH', 'AGGH')]
res = extendAln(1, 1, dicoSeq['s2'], 'AGGHV', blosum, 20, 3, 13)
print (res)
#[17, 5, ('CGGHR', 'AGGHV')]


IndexError: tuple index out of range

In [ ]:
def getWordsQuery(w, query, indexB):
    """
    Loking for indexed words in the query
    Input1 w: word size
    Input2 query: sequence
    Input3 indexB: indexed database
    Output1 foundWords: Tuple containg found words, position in query, list of pairs (Subject id, Subject position)
    """
    foundWords=[]

    return foundWords

foundWords = getWordsQuery(w,  'AGGHV', index)
print (foundWords)
#[('GGH', 1, [('s2', 1)])]


In [ ]:
#Implement BLAST algorithm

def Blast(foundWords, indexedDB, Query, dicoSeq, T, w, matriceBlosum):
    """
    input1 foundWords: Tuple containg found words, position in query, list of pairs (Subject id, Subject position
    input2 indexedDB: indexed database
    input3 query: query sequence
    input4 dicoSeq: dictionary containing database sequences
    input5 T: Threahold score
    input6 w: word size
    Input7 matriceBlosum: a dictionary containing BLOSUM matrix
    output1 alignedSeq: Tuple of aligned sequences
    """
    alignedSeq=()


    return alignedSeq

ThrScore = 12
alignedSeq = Blast(foundWords, index, "AGGHV", blosum, w, ThrScore)


<h3>Section 2: PSI-BLAST</h3>
On considère à nouveau le gène FHIT chez l'homme.
<BR>
<b> Exercice 1 </b>:
Faites un PSI-BLAST de ce gène contre la base SWISS-PROT. Utilisez un seuil pour l’évalue de 1e-5.
Que constatez-vous lors de la 1ère itération concernant les protéines de la famille des galactose-1-phosphate uridylyltransferases?


<font color="red"> Réponse : </font>
<font color="blue">
Put your answers here
</font>

<b> Exercice 2 </b>:
Que constatez-vous lors de la 2eme itération? Lors de la 5eme itération?


<font color="red"> Réponse : </font>
<font color="blue">
Put your answers here
</font>

<b> Exercice 3 </b>:
À partir de quelle itération peut-on considérer qu'il y a eu une convergence de l'algorithme PSI- BLAST? Pour quoi ?

<font color="red"> Réponse : </font>
<font color="blue">
Put your answers here
</font>

<h3>Section 3: Alignement Multiple  </h3>


<b> Exercice 1 </b>:
Accédez à l'application <a href="http://www.ebi.ac.uk/Tools/msa/clustalo/">Clustal omega</a> et copiez les 7 séquences du fichier 7sequences.fasta. Expliquez, dans quel ordre les séquences ont été alignées.

<font color="red"> Réponse : </font>
<font color="blue">
Put your answers here
</font>

<b> Exercice 2 (OPTIONAL)  </b>:
Faite une fonction pour calculer le score d’un alignement multiple. Le score de chaque position est la somme des tous les scores obtenus par BLOSUM62, et le score final est la somme des scores de chaque position divisée par la taille du alignement.

In [ ]:
#Read multi fasta
def extraireAllFastaMulti(fileName):
    """
    Read a FASTA file with several sequences
    input1 fileName: file name
    output1 IdSeq: list of sequences IDs
    output2 Seqs: sequences
    """

    Seqs=[]
    IdSeq=[]

    return IdSeq, Seqs

In [ ]:
def SP_score(SeqsM, blosum, gap):
    """
    Compute MSA score based on a substitution matrix and gap penality
    input1 SeqsM: list of aligned sequences
    input2 blosum: substitution matrix
    input3 gap: gap penality
    output1 score: MSA score
    """

    score = 0

    return (score)

SeqsM = ['ACG', 'A-G', 'ACD']
score = SP_score(SeqsM, blosum, gap)
print (score)
#5.0

<b> Exercice 3 </b>: Aligner les séquences du fichiers algnMult2.fasta avec  <a href="http://www.ebi.ac.uk/Tools/msa/clustalo/">Clustal omega</a>. Utiliser la fonction développé precedement  pour calculer le score du alignement. Quelle est la séquence la plus divergente?

<font color="red"> Réponse : </font>
<font color="blue">
Put your answers here
</font>